In [14]:
import requests
import pandas as pd
import numpy as np
import gmaps
import json
import csv
from keys import gapikey
from matplotlib.cm import winter
from matplotlib.colors import to_hex
from area import area

gmaps.configure(api_key=gapikey)
with open('zcta/zcta.json') as a:
    geoj = json.load(a)
with open('resources/zip_targets.csv', newline='\n', encoding='utf-8-sig') as b:
    ziptgt = pd.read_csv(b)
with open('resources/dc_pop.csv', newline='\n', encoding='utf-8-sig') as c:
    dc_pop = pd.read_csv(c)

In [22]:
#extract just population and zip from dc_pop.csv
dc_popdf = dc_pop[['GEO.id2','HC01_EST_VC01']]
#rename columns
dc_popdf = dc_popdf.rename(columns={'GEO.id2': 'zip', 'HC01_EST_VC01': 'population'})
#outer merge with target zipcode list
dc_maindf = ziptgt.merge(dc_popdf, on='zip', how='outer').set_index('zip')
#fill NaN's with zero
dc_maindf = zippopdf.fillna(value=0)
#create dict
zipdict = dict(dc_maindf['population'])



In [23]:
poplist = []
geoj_scrubbed = {}
areadict = {}
features = geoj['features']
feature_col = []
zip_areas = []


for feature in features:
    try:
        feature_geoid = int(feature['properties']['GEOID10'])
        if feature_geoid in zipdict.keys():
            feature_col.append(feature)
            poplist.append(zipdict[feature_geoid])
            
            zarea = area(feature['geometry'])/int(1E6)
            zip_areas.append(zarea)
            areadict.update({feature_geoid:zarea})
            
            
    except ValueError:
            print(f'Value Error {b}')
                  
geoj_scrubbed = {'type': 'FeatureCollection', 'features': feature_col}

areadf = pd.DataFrame.from_dict(areadict, orient='index', columns=['area'])

zippopdf2 = zippopdf.join(areadf, how='left')
zippopdf2['density'] = zippopdf2['population']/zippopdf2['area']



In [24]:
def calculate_color(pop):
    min_pop = min(poplist)
    max_pop = max(poplist)
    pop_range = max_pop - min_pop
    normalized_pop = ((pop - min_pop)/pop_range)
    mpl_color = winter(normalized_pop)
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    
    return gmaps_color

colors = []

for pop in poplist:
    if int(pop) > 0:
        try:
            color = calculate_color(pop)
        except KeyError:
            color = (0, 0, 0, 0.3)
    else:
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [25]:
zcta_layer = gmaps.geojson_layer(
    geoj_scrubbed,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig = gmaps.figure()
fig.add_layer(zcta_layer)
fig

Figure(layout=FigureLayout(height='420px'))